In [9]:
%%sh

# The name of our algorithm
algorithm_name=text-generation

cd container

chmod +x code/train
#chmod +x code/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

# On a SageMaker Notebook Instance, the docker daemon may need to be restarted in order
# to detect your network configuration correctly.  (This is a known issue.)
if [ -d "/home/ec2-user/SageMaker" ]; then
  sudo service docker restart
fi

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Stopping docker: [  OK  ]
Starting docker:	.[  OK  ]
Sending build context to Docker daemon  24.58kB
Step 1/6 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.0.0-cpu-py36-ubuntu18.04
2.0.0-cpu-py36-ubuntu18.04: Pulling from tensorflow-training
Digest: sha256:b3258b13d2c6333075494b8017d261a4bde5c89bdcfea3f99283274987294dac
Status: Downloaded newer image for 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.0.0-cpu-py36-ubuntu18.04
 ---> a709409e7f8c
Step 2/6 : RUN apt-get update &&     apt-get install -y nginx
 ---> Running in 03cef63635ab
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [808 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [26.1 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 P

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [10]:
# S3 prefix
prefix = 'text-generation'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [11]:
# Create session
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [12]:
# upload data for training
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [13]:
print(data_location)
print(sess.default_bucket())

s3://sagemaker-us-east-1-731833107751/text-generation
sagemaker-us-east-1-731833107751


In [14]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/text-generation:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c5.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

2020-02-03 17:59:25 Starting - Starting the training job...
2020-02-03 17:59:27 Starting - Launching requested ML instances......
2020-02-03 18:00:32 Starting - Preparing the instances for training...
2020-02-03 18:01:18 Downloading - Downloading input data...
2020-02-03 18:01:31 Training - Downloading the training image...
2020-02-03 18:02:23 Training - Training image download completed. Training in progress..2020-02-03 18:02:26,735 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-02-03 18:02:26,741 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-02-03 18:02:26,752 sagemaker-containers ERROR    Reporting training FAILURE
2020-02-03 18:02:26,752 sagemaker-containers ERROR    framework error: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_containers/_trainer.py", line 81, in train
    entrypoint()
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_tensorflow

UnexpectedStatusException: Error for Training job text-generation-2020-02-03-17-59-25-565: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_containers/_trainer.py", line 81, in train
    entrypoint()
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_tensorflow_container/training.py", line 211, in main
    train(env, framework.mapping.to_cmd_args(user_hyperparameters))
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_tensorflow_container/training.py", line 158, in train
    runner=runner_type)
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_containers/entry_point.py", line 93, in run
    install(user_entry_point, _env.code_dir, capture_error)
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_containers/entry_point.py", line 118, in install
    entrypoint_type = _entry_point_type.get(dst, name)
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_containers/_ent